### Face Liveness - Heuristic-based solution to recognize human face by detecting eye &  lip movement and blink detection. 
Uses "shape_predictor_68_face_landmarks.dat" library which plotes 68 predefine points on face.Using these points tracks eye and mouth and using distance algorithm wull check if eye is blinked or not and mouth is open or not.

In [ ]:
import cv2
import numpy as np
import dlib
import math
import imutils
import face_recognition as fr
import os
import re
import time
import pendulum
from known_face_encodings import *

In [ ]:
# Get the known face names from the disk
known_face_encodings,known_face_names = initial()
known_face_names = [re.findall(r'.*(?=[.])',a)[0] for a in known_face_names]

PATH           = "C:/Users/sriha/Documents/My_Data/Work_Documents/Posidex/face_rec/face_rec-blink/"
face_detector  = cv2.CascadeClassifier(PATH + "haarcascade_frontalface_alt.xml")
#open_eyes_detector   = cv2.CascadeClassifier(PATH + "haarcascade_eye_tree_eyeglasses.xml")
detector       = dlib.get_frontal_face_detector()
predictor      = dlib.shape_predictor(PATH + "shape_predictor_68_face_landmarks.dat")
os.environ['OPENCV_VIDEOIO_PRIORITY_MSMF'] = '0'

### Routine to determine midpoint between two coordinates

In [ ]:
def midpoint(p1 ,p2):
    return int((p1.x + p2.x)/2), int((p1.y + p2.y)/2)

### Routine to calculate the distance between two coordinates

In [ ]:
def get_distance(X,Y):
    distance = math.sqrt((X[0] - Y[0])**2 + (X[1] - Y[1])**2)
    return distance

### Routine to check for eye blinking

In [ ]:
def isBlinking(landmarks):
    
    # from 68 facial landmark coordinates   
    ##############################################
    # Eye detection                              #
    # Left eye points: (36, 37, 38, 39, 40, 41)  #
    # Right eye points: (42, 43, 44, 45, 46, 47) #
    ##############################################
  
        # Left Eye
        Left_left_point    = (landmarks.part(36).x,landmarks.part(36).y)
        Left_right_point   = (landmarks.part(39).x,landmarks.part(39).y)
        Left_center_top    = midpoint(landmarks.part(37),landmarks.part(38))
        Left_center_bottom = midpoint(landmarks.part(41),landmarks.part(40))
        Left_hor_line      = cv2.line(frame, Left_left_point, Left_right_point, (0, 255, 255), 1)
        Left_ver_line      = cv2.line(frame, Left_center_top, Left_center_bottom, (0, 255, 255), 2)
         
        # right Eye
        Right_left_point    = (landmarks.part(42).x,landmarks.part(42).y)
        Right_right_point   = (landmarks.part(45).x,landmarks.part(45).y)
        Right_center_top    = midpoint(landmarks.part(43),landmarks.part(44))
        Right_center_bottom = midpoint(landmarks.part(47),landmarks.part(46))
        Right_hor_line      = cv2.line(frame, Right_left_point, Right_right_point, (0, 255, 255), 1)
        Right_ver_line      = cv2.line(frame, Right_center_top, Right_center_bottom, (0, 255, 255), 2)
        
        
        # Get the distance of horizontal and vertical lines
        
        Left_Y_distance = get_distance(Left_center_top,Left_center_bottom)
        Left_X_distance = get_distance(Left_left_point,Left_right_point)
        Left_ratio      = Left_X_distance/Left_Y_distance
        
        Right_Y_distance = get_distance(Right_center_top,Right_center_bottom)
        Right_X_distance = get_distance(Right_left_point,Right_right_point)
        Right_ratio      = Right_X_distance/Right_Y_distance
        
        blinking_ratio = ((Left_ratio+Right_ratio)/2)
        
        return(blinking_ratio)
    

### Routine to check for mouth open

In [ ]:
def isOpen(landmarks):
    
    # from 68 facial landmark coordinates 
    ##############################################
    # Mouth detection                            #
    # Left points: (48)                          #
    # Right points: (54)                         #
    # Top Lips points: (61,63)                   #
    # Bottom Lips points: (65,67)                #
    ##############################################
  
        Mouth_left_point     = (landmarks.part(48).x,landmarks.part(48).y)
        Mouth_right_point    = (landmarks.part(54).x,landmarks.part(54).y)
        Mouth_center_top     = midpoint(landmarks.part(61),landmarks.part(63))
        Mouth_center_bottom  = midpoint(landmarks.part(67),landmarks.part(65))
        hor_line             = cv2.line(frame, Mouth_left_point, Mouth_right_point, (0, 255, 255), 1)
        ver_line             = cv2.line(frame, Mouth_center_top, Mouth_center_bottom, (0, 255, 255), 2)
         
        # Get the distance of horizontal and vertical lines
        
        Y_distance = get_distance(Mouth_center_top,Mouth_center_bottom)
        X_distance = get_distance(Mouth_left_point,Mouth_right_point)
        
        try:
            open_ratio = float(Y_distance)/float(X_distance)
        except ZeroDivisionError:
            open_ratio = 0
        
        return(open_ratio)
    

### Routine to check for nose moment

In [7]:
def noseMove(landmarks):
    
    # from 68 facial landmark coordinates 
    ##############################################
    # Nose  detection                            #
    # Left points: (31)                          #
    # Right points: (35)                         #
    ##############################################
  
        Nose_left_point     = (landmarks.part(31).x,landmarks.part(31).y)
        Nose_right_point    = (landmarks.part(35).x,landmarks.part(35).y)
        hor_line            = cv2.line(frame, Nose_left_point, Nose_right_point, (0, 255, 255), 1)
         
        # Get the distance of horizontal and vertical lines
        
        X = Nose_right_point[0] - Nose_left_point [0] 
        Y = Nose_left_point [1] - Nose_right_point[1]
        
        if (Y < 0):
            Nose_movement = 'Y'
        else:
            Nose_movement = 'N'

        return(Nose_movement)
    

In [8]:
def faceMove(landmarks):
    
    # from 68 facial landmark coordinates 
    ##############################################
    # Face  detection                            #
    # Left points: (1)                          #
    # Right points: (15)                         #
    ##############################################
  
        Face_left_point     = (landmarks.part(1).x,landmarks.part(1).y)
        Face_right_point    = (landmarks.part(15).x,landmarks.part(15).y)
        hor_line            = cv2.line(frame, Face_left_point, Face_right_point, (0, 255, 255), 1)
         
        # Get the distance of horizontal and vertical lines
        
        X = Face_right_point[0] - Face_left_point [0] 
        Y = Face_left_point [1] - Face_right_point[1]
        
        if (Y < 0):
            Face_movement = 'Y'
        else:
            Face_movement = 'N'
            
        return(Face_movement)
    

### Compare Live face with known face encodings and get identity having the closest distance

In [9]:
# Check is unknown face is a match in the known face(s)
# if distance bwtween unknown and known face is <= tolerance is a match

def getIdentity(unknown_image,Upsample):
    
    global result 
    global Known_face_match
    global min_distance
    
    name               = "NoMatch"
    
    # resize, convert and align
    unknown_image   = imutils.resize(unknown_image, width=400)
    gray            = cv2.cvtColor(unknown_image, cv2.COLOR_BGR2GRAY) 
    clahe           = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(5,8)) # Divide image into small blocks called “tiles” (Size is 8x8 by default in OpenCV)
    gray            = clahe.apply(gray)
    gray            = cv2.bilateralFilter(gray,5,75,75) # bilateral filter d = 15 and sigmaColor = sigmaSpace = 75. The greater its value, the more further pixels will mix together, given that their colors lie within the sigmaColor range
    rects           = hog_face_detector(gray, Upsample)
    
    if rects:
        
        # for each detected face
        for rect in rects:
            unknown_image          = fa.align(unknown_image, gray, rect)
            face_locations         = fr.face_locations(unknown_image,model="hog")   
            #face_locations         = fr.face_locations(unknown_image,model="cnn")   
            unknown_face_encodings = fr.face_encodings(unknown_image, face_locations) # Encoding one face since each image only has one face
            
        for (top, right, bottom, left), unknown_face_encoding in zip(face_locations, unknown_face_encodings):

            matches          = fr.compare_faces(known_face_encodings, unknown_face_encoding,tolerance=0.6)
            distance         = fr.face_distance(known_face_encodings, unknown_face_encoding)
            distance         = list(np.around(np.array(distance),2))
            best_match_index = np.argmin(distance)   # Get smallest distance between the two faces
            min_distance     = distance[best_match_index]
            
            if matches[best_match_index]:
                result           = 'Match '
                Known_face_match = known_face_names[best_match_index]   
            else:
                result            = "NoMatch"
                Known_face_match  = "UnKnown_Face"
        
    else:
        result            = "NoMatch"
        Known_face_match  = "UnKnown_Face"
        min_distance      = 0 
        

    return result,Known_face_match

### Get the Live Image

In [19]:
mopen = 'N'
blink = 'N'
start = 'N'
blinkRatio = 0
mouthRatio = 0
overlay       = cv2.imread('C:/Users/sriha/Desktop/blink.jpg')
overlay       = cv2.resize(overlay,(800, 500))
video         = cv2.VideoCapture(0)  # Read the image from camera

while True:
    
    success, frame  = video.read()
    if not success: 
        print('error')
        break
        
    frame   = cv2.resize(frame, (800, 500))
    gray    = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces   = detector(gray)
    persons = face_detector.detectMultiScale(gray,scaleFactor=1.3,minNeighbors =5,minSize=(50, 50),flags=cv2.CASCADE_SCALE_IMAGE) 
    
    for person in persons:
        (x, y, w, h) = person
            
        for face in faces:
            landmarks     = predictor(gray,face)
            mouthRatio    = isOpen(landmarks)
            blinkRatio    = isBlinking(landmarks)
            noseMovement  = noseMove(landmarks)
            faceMovement  = faceMove(landmarks)
            

            if (blinkRatio > 6.2):
                cv2.putText(frame, 'Blink detected', (100,112), cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,0), 2)
                blink = 'Y'

            if (mouthRatio > 0.2):
                cv2.putText(frame, 'Mouth open', (100,112), cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,0), 2)
                mopen = 'Y'
            
            if (noseMovement == 'Y'):
                cv2.putText(frame, 'Nose Movement', (100,112), cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,0), 2)
            
            if (faceMovement == 'Y'):
                cv2.putText(frame, 'Face Movement', (100,112), cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,0), 2)
            
            
            cv2.putText(frame,pendulum.now().to_day_datetime_string(),(550,20), cv2.FONT_HERSHEY_PLAIN,1,(0,0,0),2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            #if ((blink == 'Y') or (mopen=='Y') or (noseMovement == 'Y') or (faceMovement == 'Y')):
            if ((blink == 'Y') or (mopen=='Y') or (noseMovement == 'Y')):
                
                if (start=='N'):
                    starttime        = time.time()
                    result,knownface = getIdentity(frame,Upsample=1)
                    start='Y'
                
                y = y - 15 if y - 15 > 15 else y + 15
                #frame  = cv2.resize(frame, (800, 500))
                output = cv2.addWeighted(frame, 0.7, overlay, 0.3, 0)
                cv2.putText(output, knownface, (x, y), cv2.FONT_HERSHEY_SIMPLEX,1, (0, 0, 255), 3)
                cv2.putText(output, 'Congratulations.Your new Image accepted.', (40,70), cv2.FONT_HERSHEY_SIMPLEX,1, (0, 0, 0), 3)
                cv2.imshow('Face', output)
                
            else:
                cv2.putText(frame, 'UnKnown', (x, y-5), cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 3)
                cv2.imshow('Face', frame)

    if ((blink == 'Y') or (mopen=='Y') or (noseMovement == 'Y')):
        if ((time.time()-starttime) > 10):
            break
    
    if (cv2.waitKey(1)==13):   # 13 is ASCII code for Enter Key
        break

video.release()
cv2.destroyAllWindows()

In [ ]:
### Donot delete

In [ ]:
mopen = 'N'
blink = 'N'
start = 'N'
overlay       = cv2.imread('C:/Users/sriha/Desktop/blink.jpg')
overlay       = cv2.resize(overlay,(800, 500))
video         = cv2.VideoCapture(0)  # Read the image from camera

while True:
    
    success, frame  = video.read()
    if not success: 
        print('error')
        break
        
    frame   = cv2.resize(frame, (800, 500))
    gray    = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces   = detector(gray)
    persons = face_detector.detectMultiScale(gray,scaleFactor=1.3,minNeighbors =5,minSize=(50, 50),flags=cv2.CASCADE_SCALE_IMAGE) 
    
    for person in persons:
        (x, y, w, h) = person
            
        for face in faces:
            landmarks   = predictor(gray,face)
            mouthRatio  = isOpen(landmarks)
            blinkRatio  = isBlinking(landmarks)

            if (blinkRatio > 5.7):
                cv2.putText(frame, 'Blinking', (80,112), cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,0), 2)
                blink = 'Y'

            if (mouthRatio > 0.2):
                cv2.putText(frame, 'OPEN', (100,112), cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,0), 2)
                mopen = 'Y'
             
            cv2.putText(frame,pendulum.now().to_day_datetime_string(),(390,20), cv2.FONT_HERSHEY_PLAIN,1,(255,255,255),2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            if ((blink == 'Y') or (mopen=='Y')):
                
                if (start=='N'):
                    starttime        = time.time()
                    result,knownface = getIdentity(frame,Upsample=1)
                    start='Y'
                
                y = y - 15 if y - 15 > 15 else y + 15
                #frame  = cv2.resize(frame, (800, 500))
                output = cv2.addWeighted(frame, 0.7, overlay, 0.3, 0)
                cv2.putText(output, 'Name:'+ knownface, (x, y), cv2.FONT_HERSHEY_SIMPLEX,1, (0, 255, 0), 2)
                cv2.putText(output, 'Congratulations.Your new Image accepted.', (40,135), cv2.FONT_HERSHEY_SIMPLEX,1, (255,255,255), 3)
                cv2.imshow('Face', output)
                
            else:
                cv2.putText(frame, 'Name:UnKnown', (x, y-5), cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
                cv2.imshow('Face', frame)

    if ((blink == 'Y') or (mopen=='Y')):
        if ((time.time()-starttime) > 15):
            break
    
    if (cv2.waitKey(1)==13):   # 13 is ASCII code for Enter Key
        break

video.release()
cv2.destroyAllWindows()

In [ ]:
        
            
    
    
    img,blink,mopen = show(video,mopen)
    
    if ((time.time()-starttime) > 5):
        cv2.putText(img, 'App will Close in 10 sec', (80,112), cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,0), 2)
    if ((time.time()-starttime) > 35):
        break
    
    cv2.putText(img,pendulum.now().to_day_datetime_string(),(390,20), cv2.FONT_HERSHEY_PLAIN,1,(255,255,255),2)
    img  = cv2.resize(img, (800, 500))
    
    if (blink == 'Y') or (mopen == 'Y'):
        
        output  = cv2.addWeighted(img, 0.7, overlay, 0.3, 0)
        cv2.imshow('Face', output)
    else:
        cv2.imshow('Face', img)
    
    if (cv2.waitKey(1)==13):   # 13 is ASCII code for Enter Key
        break


In [ ]:
PATH                 = "C:/Users/sriha/Documents/My_Data/Work_Documents/Posidex/face_rec/face_rec-blink/"
os.environ['OPENCV_VIDEOIO_PRIORITY_MSMF'] = '0'
cascPath  = PATH + "haarcascade_frontalface_default.xml"
eyePath   = PATH + "haarcascade_eye.xml"
smilePath = PATH + "haarcascade_smile.xml"

faceCascade  = cv2.CascadeClassifier(cascPath)
eyeCascade   = cv2.CascadeClassifier(eyePath)
smileCascade = cv2.CascadeClassifier(smilePath)

font = cv2.FONT_HERSHEY_SIMPLEX
video_capture = cv2.VideoCapture(0)

while True:
    
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = faceCascade.detectMultiScale(gray,scaleFactor=1.1, minNeighbors=5,minSize=(30, 30),flags=cv2.CASCADE_SCALE_IMAGE)

    for (x, y, w, h) in faces:
        if w > 250 :
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 3)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            cv2.putText(frame,'Face',(x, y), font, 2,(255,0,0),5)
            
    smile = smileCascade.detectMultiScale(roi_gray,scaleFactor= 1.16,minNeighbors=35,minSize=(25, 25),flags=cv2.CASCADE_SCALE_IMAGE)
    for (sx, sy, sw, sh) in smile:
        cv2.rectangle(roi_color, (sh, sy), (sx+sw, sy+sh), (255, 0, 0), 2)
        cv2.putText(frame,'Smile',(x + sx,y + sy), 1, 1, (0, 255, 0), 1)
        
    eyes = eyeCascade.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
        cv2.putText(frame,'Eye',(x + ex,y + ey), 1, 1, (0, 255, 0), 1)
    
    cv2.putText(frame,'Number of Faces : ' + str(len(faces)),(40, 40), font, 1,(255,0,0),2) 
    cv2.imshow('Video', frame)
    
    if (cv2.waitKey(1)==13):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()